In [1]:
import os
import math
import torch
import torchvision
import collections
import numpy as np
import pandas as pd
import cv2
from math import exp
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms, datasets
from skimage.io import imread, imsave
from skimage import transform
from skimage.transform import rotate, AffineTransform
from skimage.io import imread
from skimage.color import rgb2gray, gray2rgb
from scipy import ndimage
import skimage.io as io
from PIL import Image

In [ ]:
x_train= ['{}'.format(num) for num in range(1,4)]
y_train= ['{}'.format(num) for num in range(1,4)]


In [ ]:
train=pd.DataFrame()
train['x_train']=x_train
train['y_train']=y_train

In [ ]:
class Super_resol_Dataset():
    
    def __init__(self, dataset, root_dir_x, root_dir_y, transform1=None, transform2=None):

        self.data = dataset
        self.root_dir_x = root_dir_x
        self.root_dir_y = root_dir_y
        self.transform1 = transform1
        self.transform2 = transform2

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx

        img_name = os.path.join(self.root_dir_x,self.data.iloc[idx, 0])
        
        image = Image.open(img_name+".png")
        label_name = os.path.join(self.root_dir_y,self.data.iloc[idx, 1])
        
        label = Image.open(label_name+".png")        
        if self.transform1 is not None:
            image = self.transform1(image)
        if self.transform2 is not None:
            label = self.transform2(label)
        return image, label
    


In [ ]:
batch_size=1
train_transform = transforms.Compose([transforms.Resize((1024,1024)),transforms.ToTensor()])
test_transform = transforms.Compose([transforms.Resize((1024,1024)),transforms.ToTensor()])

trainset = Super_resol_Dataset(dataset=train,root_dir_x='/content/sem/image/',
                               root_dir_y='/content/sem/label/',transform1=train_transform,transform2=test_transform)


In [5]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        


In [6]:
createFolder('/content/sem_128/')


In [ ]:
createFolder('/content/sem_128/image/')
createFolder('/content/sem_128/label/')


In [ ]:
# Dividing input image into 128 patch size
size = 128
stride = 128

k=1
path1='/content/sem_128/image/'
path2='/content/sem_128/label/'
for i,data in enumerate(trainset):
  inputs,labels = data
  inputs=inputs.contiguous().view(inputs.size(0), inputs.size(1),inputs.size(2))
  labels=labels.contiguous().view(labels.size(0), labels.size(1),labels.size(2))
  inputs = inputs.unfold(1, size, stride).unfold(2, size, stride)
  labels = labels.unfold(1, size, stride).unfold(2, size, stride)
  inputs = inputs.contiguous().view(inputs.size(0), -1, size, size)
  labels = labels.contiguous().view(labels.size(0), -1,size, size)
  inputs =inputs.permute(1,0,2,3)
  labels =labels.permute(1,0,2,3)
  for i in range(inputs.shape[0]):
    p=inputs[i,:]
    im = transforms.ToPILImage()(p)
    im.save(path1+"{}".format(k)+".png")
    p=labels[i,:]
    im = transforms.ToPILImage()(p)
    im.save(path2+"{}".format(k)+".png")
    k+=1
  


In [ ]:
k

13

In [ ]:
#Augmenting image by rotating into different angle
s=k
for i in range(1,s):
  p=str(i)
  inputs=io.imread(path1+p+".png")
  labels=io.imread(path2+p+".png")
  rot=[30,45,60,90]
  for j in rot:
    rotated_in = rotate(inputs, angle=j, mode = 'wrap')
    rotated_out = rotate(labels, angle=j, mode = 'wrap')
    rotated_in  = (rotated_in * 255).astype(np.uint8)
    rotated_out  = (rotated_out * 255).astype(np.uint8)

    p = torch.from_numpy(rotated_in)
    im = transforms.ToPILImage()(p)
    im.save(path1+"{}".format(k)+".png")

    p = torch.from_numpy(rotated_out)
    im = transforms.ToPILImage()(p)
    im.save(path2+"{}".format(k)+".png")
    k+=1


In [ ]:
k

65

In [ ]:
#Augmenting image by flipping in horizontal and vertical direction

s=k
for i in range(1,s):
  p=str(i)
  inputs=io.imread(path1+p+".png")
  labels=io.imread(path2+p+".png")
  flipLR1 = np.fliplr(inputs).astype(np.uint8)
  flipLR2 = np.fliplr(labels).astype(np.uint8)
  flipUD1 = np.flipud(inputs).astype(np.uint8)
  flipUD2 = np.flipud(labels).astype(np.uint8)

  p = torch.from_numpy(flipLR1)
  im = transforms.ToPILImage()(p)
  im.save(path1+"{}".format(k)+".png")

  p = torch.from_numpy(flipLR2)
  im = transforms.ToPILImage()(p)
  im.save(path2+"{}".format(k)+".png")
  k+=1

  p = torch.from_numpy(flipUD1)
  im = transforms.ToPILImage()(p)
  im.save(path1+"{}".format(k)+".png")

  p = torch.from_numpy(flipUD2)
  im = transforms.ToPILImage()(p)
  im.save(path2+"{}".format(k)+".png")
  k+=1


In [ ]:
k

181

In [9]:
import shutil
shutil.make_archive('train', 'zip', '/content/sem_128')

'/content/test.zip'